Early simple rule-based model

imports

In [ ]:
def separate(sentence): 
  #takes in a sentence, splits it into the subsentences
  tokens = sentence.split() # or  nltk.word_tokenize(sentence)
  current = []
  sentences = []
  for i, word in enumerate(tokens, 0):
    if word in ["then", "followed"]:
      sentences.append(' '.join(current))
      current = []
    elif word == "and" and tokens[i+1] in ['a', 'an', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']:
      sentences.append(' '.join(current))
      current = []
    else:
      current.append(word)
    sentences.append(' '.join(current))
    return sentences


Neural network based model

This is based on the code we used in the sentence to waypoint models

Imports

In [ ]:
!pip install transformers
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelWithLMHead
from matplotlib import pyplot as plt
#from transformers import BertModel, BertConfig
from google.colab import files
files.upload()  # load dataGeneration.py

from splittingDataGenerator import generateTestBatch

     |████████████████████████████████| 1.5MB 12.5MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
     |████████████████████████████████| 2.9MB 51.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0b0514c28a7edb14ca2884ff142ce2413211e6b47d942ec2b4cd5f28ced1a081
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Saving splittingDataGenerator.py to splittingDataGenerator.py


In [ ]:
# import the pretrained models
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelWithLMHead.from_pretrained("bert-base-uncased")
baseModel = model.bert
#config = BertConfig()
#baseModel = BertModel(config)

# helper for padding 2D lists into tensors
def listToTensor(list):
  maxLen = max(len(l) for l in list)
  tensor = torch.tensor([l + (maxLen - len(l)) * [0] for l in list])
  return tensor


/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:890: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# custom head for BERT
# credit to Martin Zablocki: https://zablo.net/blog/post/custom-classifier-on-bert-model-guide-polemo2-sentiment-analysis/
class SplittingNet(nn.Module):
  def __init__(self, base_model, n_classes=30, base_model_output_size=768, hidden_size = 100, dropout=0.05):
    super().__init__()
    #self.base_model = base_model
    self.bert = base_model
    #self.linear = nn.Linear(hidden_size, 30)
    #self.dropout = nn.Dropout(dropout)
    self.lstm = nn.LSTM(input_size=base_model_output_size, hidden_size=hidden_size,
                        num_layers=2, dropout=dropout)
    #self.lstm2 = nn.LSTM(input_size)
    self.linear = nn.Linear(100, 70)
    self.activation = nn.PReLU()
    self.linear2 = nn.Linear(70, 30)
    self.activation2 = nn.PReLU()
    self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(base_model_output_size, hidden_size),
            nn.PReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, n_classes),
            nn.PReLU()
        )
    
    for layer in self.classifier:
            if isinstance(layer, nn.Linear):
                layer.weight.data.normal_(mean=0.0, std=0.02)
                if layer.bias is not None:
                    layer.bias.data.zero_()

  def forward(self, input, *args):
    X, attention_mask = input
    hidden_states = self.bert(X, token_type_ids=None, attention_mask = attention_mask)
    #print(hidden_states.last_hidden_state.shape)
    
    #output = self.classifier(hidden_states.last_hidden_state[:,0,:])
    output = self.lstm(hidden_states.last_hidden_state.permute(1,0,2))[0].permute(1,0,2)
    #print(output.shape)
    output = self.linear(output[:,-1,:])
    output = self.activation(output)
    output = self.linear2(output)
    output = self.activation2(output)
    output = output/torch.norm(output)
    #print(output.shape)
    return output
    

  #def forward(self, input, *args):
  #  X, attention_mask = input
  #  hidden_states = self.bert(X, attention_mask = attention_mask)[0]
  #  lstm_out = self.lstm(hidden_states.permute(1,0,2))[0].permute(1,0,2)
  #  return self.linear(lstm_out[:, -1, :])

This is where it trains

In [ ]:
# training model
if torch.cuda.is_available():  
  #device = torch.device("cpu")
  device = torch.device("cuda:0") 
else:  
  device = torch.device("cpu")
print(device)

net = SplittingNet(baseModel)
net = net.to(device=device)
#criterion = nn.MultiLabelSoftMarginLoss()
criterion = nn.CrossEntropyLoss()
objectiveValues = []
running_loss = 0

cuda:0


In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
# each epoch is a newly generated batch of test data
for epoch in range(5000):
  text, labels = generateTestBatch(100)
  labels = torch.tensor(labels, dtype = int)

  enc = tokenizer.batch_encode_plus(text)
  X = listToTensor(enc["input_ids"])
  attn = listToTensor(enc["attention_mask"])

  X = X.to(device)
  attn = attn.to(device)
  labels = labels.to(device)
  #labels=labels.to(torch.float32)
  #labels.type(torch.LongTensor)
  # zero the parameter gradients
  optimizer.zero_grad()

  # forward + backward + optimize
  outputs = net((X, attn))
  '''if epoch == 0:
    print(outputs.shape)
    print(labels.shape)'''
  #print(outputs.shape)
  #outputs.type(torch.LongTensor)
  #print(labels.shape)

  # For MultiLabelSoftMarginLoss
  #loss = criterion(outputs, labels)
  # For CrossEntropyLoss
  loss = criterion(outputs, torch.max(labels, 1)[1])

  #loss.type(torch.LongTensor)
  loss=loss.to(torch.float32)
  loss.backward()
  optimizer.step()

  # print statistics
  running_loss += loss.item()
  if ((epoch + 1) % 10 == 0):
    print('[%d] loss: %.4f' % (epoch + 1, running_loss / 100))
    running_loss = 0
  objectiveValues.append(loss.item())

plt.plot(objectiveValues)
plt.title("Training Loss for Sentence Splitting")
plt.xlabel("Epochs")
plt.ylabel("Cross Entropy Loss")

torch.Size([101, 30])
torch.Size([101, 30])
[10] loss: 0.3374
[20] loss: 0.3357
[30] loss: 0.3355
[40] loss: 0.3354
[50] loss: 0.3350
[60] loss: 0.3347
[70] loss: 0.3337
[80] loss: 0.3326
[90] loss: 0.3318
[100] loss: 0.3314
[110] loss: 0.3311
[120] loss: 0.3308
[130] loss: 0.3306
[140] loss: 0.3306
[150] loss: 0.3305
[160] loss: 0.3304


KeyboardInterrupt: ignored

In [ ]:
for epoch in range(10000):
  text, labels = generateTestBatch(100)
  labels = torch.tensor(labels, dtype = int)

  enc = tokenizer.batch_encode_plus(text)
  X = listToTensor(enc["input_ids"])
  attn = listToTensor(enc["attention_mask"])

  X = X.to(device)
  attn = attn.to(device)
  labels = labels.to(device)
  #labels=labels.to(torch.float32)
  #labels.type(torch.LongTensor)
  # zero the parameter gradients
  optimizer.zero_grad()

  # forward + backward + optimize
  outputs = net((X, attn))
  if epoch == 0:
    print(outputs.shape)
    print(labels.shape)
  #print(outputs.shape)
  #outputs.type(torch.LongTensor)
  #print(labels.shape)
  loss = criterion(outputs, labels)
  #loss.type(torch.LongTensor)
  loss=loss.to(torch.float32)
  loss.backward()
  optimizer.step()

  # print statistics
  running_loss += loss.item()
  if ((epoch + 1) % 100 == 0):
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / 100))
    running_loss = 0
  objectiveValues.append(loss.item())

plt.plot(objectiveValues)

In [ ]:
# test the output of the network

# helper to strip first n words from sentence
def removeWords(text, n):
  words = text.split(" ")
  if len(words) <= n:
    return ""
 
  i = sum([len(words[j]) for j in range(n)]) + n
  return text[i:]

with torch.no_grad():
  text = ["go 1 meter forward and to the right then turn left followed by moving 2 and a half feet up and to the right"]
  # text = ["turn right followed by going back 1 foot then turn left"]
  continueSplitting = True

  while continueSplitting:
    enc = tokenizer.batch_encode_plus(text)
    X = listToTensor(enc["input_ids"])
    attn = listToTensor(enc["attention_mask"])
    X = X.to(device)
    attn = attn.to(device)
    output = net((X, attn))

    print(text[0])
    print(output)

    n = torch.argmax(output).item()
    text = [removeWords(text[0], n)]

    print(n)
    print(text[0], "\n")

    # replace this with a better stopping condition once the network is working
    if text[0] == "":
      continueSplitting = False


go 1 meter forward and to the right then turn left followed by moving 2 and a half feet up and to the right
tensor([[-0.0307, -0.0362, -0.0252, -0.0221, -0.0355, -0.0344, -0.0231, -0.0141,
          0.9882, -0.0220, -0.0293, -0.0041, -0.0114, -0.0223, -0.0286, -0.0137,
         -0.0392, -0.0274, -0.0462, -0.0178, -0.0293, -0.0297, -0.0296, -0.0280,
         -0.0253, -0.0365, -0.0343, -0.0198, -0.0340, -0.0333]],
       device='cuda:0')
8
then turn left followed by moving 2 and a half feet up and to the right 

then turn left followed by moving 2 and a half feet up and to the right
tensor([[-0.0981, -0.0534,  0.5341,  0.7375, -0.0438, -0.0579, -0.0981, -0.0773,
         -0.0399, -0.0457, -0.0291, -0.0630, -0.0624, -0.0356, -0.0652, -0.0630,
         -0.0704, -0.1060, -0.0466, -0.0674, -0.0983, -0.1174, -0.1029, -0.0341,
         -0.0421, -0.0594, -0.0701, -0.1162, -0.1455, -0.1122]],
       device='cuda:0')
3
followed by moving 2 and a half feet up and to the right 

followed by moving 

In [ ]:
# save model
torch.save(net.state_dict(), "splittingBERT2.pth")
files.download("splittingBERT2.pth")

In [ ]:
# load model
torch.load(net.state_dict(), "splittingBERT2.pth")

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
#file_id = 'REPLACE_WITH_YOUR_FILE_ID'
#downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp splittingBERT.pth /content/drive/MyDrive